Run the below in bash

`tensorboard --logdir=./results`

In [1]:
from ray import train, tune
from ray.rllib.algorithms.ppo import PPOConfig
import os
from pathlib import Path
import gymnasium as gym
import numpy as np
import torch
from ray.rllib.core.rl_module import RLModule
from ray.tune.registry import register_env
from gymnasium.envs.box2d import LunarLander

In [2]:
class CustomLunarLander(LunarLander):
    def __init__(self, gravity=-9.81, wind_power=15.0, enable_wind=False, continuous=True, **kwargs):
        self.custom_gravity = gravity
        self.custom_wind_power = wind_power
        self.custom_enable_wind = enable_wind

        # Pass the continuous flag correctly
        super().__init__(continuous=continuous, **kwargs)

    def reset(self, **kwargs):
        # Inject custom parameters before calling base reset
        self.world.gravity = (0, self.custom_gravity)
        self.wind_power = self.custom_wind_power
        self.enable_wind = self.custom_enable_wind

        return super().reset(**kwargs)

register_env("ContinuousLunarLander", CustomLunarLander)

In [ ]:

config = (
    PPOConfig()
    .environment("LunarLanderContinuous-v3")
    # Specify a simple tune hyperparameter sweep.
    .training(
        lr=0.0005,
        lambda_=0.99,
        grad_clip=0.2,
        train_batch_size = 10000,
        minibatch_size=512,
        num_epochs = 10,
    )
    .learners(
        num_learners=15,  # Set this to the desired number of remote Learner actors
        num_cpus_per_learner=1,
        num_gpus_per_learner=0 # Set this to 1 if using GPUs, or 0 for CPU-only training
    )
)


In [4]:

# Create a Tuner instance to manage the trials.
tuner = tune.Tuner(
    config.algo_class,
    param_space=config,
    run_config=train.RunConfig(
        storage_path=os.path.abspath("./results/LundarLander"),
        stop={"env_runners/episode_return_mean": 250.0,
              "training_iteration": 5000},
        checkpoint_config=tune.CheckpointConfig(checkpoint_frequency=10,
                                                checkpoint_at_end=True,
        ),
    ),
)
# Run the Tuner and capture the results.
results = tuner.fit()

2025-09-27 16:23:26,557	WARNING algorithm_config.py:5045 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
(PPO pid=298123) 2025-09-27 16:23:29,483	WARNING algorithm_config.py:5045 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
(PPO pid=298123) [2025-09-27 16:23:29,801 E 298123 298123] core_worker.cc:2246: Actor with class name: 'SingleAgentEnvRunner' and ID: '633dcf8c75875230f28746

(_WrappedExecutable pid=298547) [Gloo] Rank 2 is connected to 14 peer ranks. Expected number of connected peer ranks is : 14


(PPO(env=LunarLanderContinuous-v3; env-runners=2; learners=15; multi-agent=False) pid=298123) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/media/ash/Storage/Projects/RL/results/LundarLander/PPO_2025-09-27_16-23-24/PPO_LunarLanderContinuous-v3_96592_00000_0_2025-09-27_16-23-26/checkpoint_000000)
(_WrappedExecutable pid=298548) 2025-09-27 16:23:37,169	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future! [repeated 17x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(PPO(env=LunarLanderContinuous-v3; env-runners=2; learners=15; multi-agent=False) pid=298123) Checkpoint succ